# Utilizando o BERTimbau para criação dos vetores

https://huggingface.co/neuralmind/bert-base-portuguese-cased

In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import psycopg2 as pgsql
import pandas as pd
import requests

In [2]:
load_dotenv()

True

In [3]:
user=os.environ['user']
password=os.environ['password']
host=os.environ['host']
port=os.environ['port']
database=os.environ['database']

url = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
con = create_engine(url)

In [4]:
# Selecionando todos os campos textuais que serão utilizados para construir o vetor do produto

query_infos = """
    SELECT
        f.medicamento_id,
        CONCAT(clf.produto, ' ', clf.principio_ativo, ' ', 
            clf.tipo, ' ', clf.categoria, ' ', 
            clf.classe_terapeutica, ' ', clf.especialidade, ' ', 
            clf.fabricante) as descricao
    FROM fat_produto f
    INNER JOIN dim_classificacao_produto clf
        on f.medicamento_id = clf.medicamento_id
    WHERE
        1=1
        and f.deleted_at is null
"""

In [5]:
df = pd.read_sql(query_infos, con)

In [6]:
df.head()

,medicamento_id,descricao
0,1,A Curitybina Ácido Salicílico Similar Calos E ...
1,2,A Saúde da Mulher Plumeria Lancifolia + Salici...
2,3,Advantage Max3 para Cães Imidacloprida + Perme...
3,4,Timosan Maleato De Timolol Similar Doenças Dos...
4,5,Timosopt Cloridrato De Dorzolamida + Maleato D...


In [7]:
# utilizando a API de inferencia do hugging face

model_id = "neuralmind/bert-base-portuguese-cased"
hf_token = os.environ['hf_token']

api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
headers = {"Authorization": f"Bearer {hf_token}"}

def query(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

In [8]:
df_teste = df[:5].copy() # fazendo um teste para as 5 primeiras linhas
output = query(df_teste['descricao'].tolist())

In [9]:
vector_list = []
for i in range(0, len(output)):
    vector_list.append(output[i][0][0]) # coletando apenas o token CLS

df_teste['vetor'] = vector_list

In [10]:
df_teste

,medicamento_id,descricao,vetor
0,1,A Curitybina Ácido Salicílico Similar Calos E ...,"[0.5235412120819092, 0.019545389339327812, 0.5..."
1,2,A Saúde da Mulher Plumeria Lancifolia + Salici...,"[-0.026552362367510796, -0.29817309975624084, ..."
2,3,Advantage Max3 para Cães Imidacloprida + Perme...,"[0.19392092525959015, 0.16702786087989807, 1.4..."
3,4,Timosan Maleato De Timolol Similar Doenças Dos...,"[0.33091333508491516, 0.08722129464149475, 0.7..."
4,5,Timosopt Cloridrato De Dorzolamida + Maleato D...,"[0.311635822057724, 0.0001349445665255189, 0.5..."


In [11]:
len(df_teste['vetor'][0]) # dimensão do vetor

768

### Criando tabela de vetores e populando com o teste

In [12]:
def exec_query(sql):
    conn = pgsql.connect(database = database, 
                     user = user, 
                     host= host,
                     password = password,
                     port = port)
    
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()

In [13]:
# criando a tabela de vetores
sql_create = """CREATE TABLE dim_vetores_teste (
            medicamento_id bigint PRIMARY KEY, 
            vetor vector(768)
            );
            """

exec_query(sql_create)

In [14]:
# populando a tabela de vetores

for n in range(0, len(df_teste)):
    sql_insert = f""" 
    INSERT INTO dim_vetores_teste(medicamento_id, vetor) VALUES({df_teste['medicamento_id'][n]}, '{df_teste['vetor'][n]}');
    """
    exec_query(sql_insert)

### Buscando por proximidade simulando um campo de busca

In [15]:
df_teste['descricao']

0    A Curitybina Ácido Salicílico Similar Calos E ...
1    A Saúde da Mulher Plumeria Lancifolia + Salici...
2    Advantage Max3 para Cães Imidacloprida + Perme...
3    Timosan Maleato De Timolol Similar Doenças Dos...
4    Timosopt Cloridrato De Dorzolamida + Maleato D...
Name: descricao, dtype: object

In [16]:
vetor = query("medicamento para caes")[0][0] # formulando um texto de busca que faça sentido com as descricoes para visualizar as opções mais proximas

query_find = f"SELECT medicamento_id FROM dim_vetores_teste ORDER BY vetor <=> '{vetor}' LIMIT 2;"

In [17]:
conn = pgsql.connect(database = database, 
                    user = user, 
                    host= host,
                    password = password,
                    port = port)

cur = conn.cursor()
cur.execute(query_find)
rows = cur.fetchall()
conn.commit()
conn.close()
for row in rows:
    print(row)

(4,)
(3,)
